In [51]:
import requests
import os

In [52]:
# Load .env environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [74]:
api_domain = "https://api.ethplorer.io"
ethplorer_api_key = "freekey"
top_tokens_to_watch = 1
top_holders_to_watch = 5
whale_alert_threshold = 100

In [75]:
# top tokens by trade volume
top_token_request_url = f"{api_domain}/getTopTokens?apiKey={ethplorer_api_key}"
top_tokens_by_trade_volume = requests.get(top_token_request_url).json().get('tokens')[:top_tokens_to_watch]

# get a map of token name -> address
token_name_to_address = {token['name']: token['address'] for token in top_tokens_by_trade_volume}
token_name_to_address

{'WETH': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'}

In [76]:
top_holder_addresses = {}

# for each token
for token, address in token_name_to_address.items():
    # find the top holders
    top_holders_request_url = f"{api_domain}/getTopTokenHolders/{address}?apiKey={ethplorer_api_key}&limit={top_holders_to_watch}"
    top_holders = requests.get(top_holders_request_url).json().get('holders')
    if top_holders:
        top_holder_addresses[token] = [holder['address'] for holder in top_holders]
        
top_holder_addresses

{'WETH': ['0x2f0b23f53734252bda2277357e97e1517d6b042a',
  '0x030ba81f1c18d280636f32af80b9aad02cf0854e',
  '0xf04a5cc80b1e94c69b48f5ee68a08cd2f09a7c3e',
  '0x94b0a3d511b6ecdb17ebf877278ab030acb0a878',
  '0x795065dcc9f64b5614c407a6efdc400da6221fb0']}

In [77]:
!pip install twilio

In [78]:
from twilio.rest import Client

In [79]:
# Your Account Sid and Auth Token from twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = os.getenv('TWILIO_ACCOUNT_SID')
auth_token = os.getenv('TWILIO_AUTH_TOKEN')
client = Client(account_sid, auth_token)

In [88]:
timestamp_offset = {}

for token, addresses in top_holder_addresses.items():
    for addr in addresses:
        request_url = f"{api_domain}/getAddressTransactions/{addr}?apiKey={ethplorer_api_key}&limit=1000"
        if addr in timestamp_offset:
            request_url += f"&timestamp={timestamp_offset}"

            
        response = requests.get(request_url).json()
        
        # continue if we run into rate limit errors
        if not isinstance(response, list):
            # print error for debugging purposes
            print(response)
            continue
            
        # update timestamp
        if response:
            timestamp_offset[addr] = max(response, key=lambda txn: txn['timestamp'])
            
        # only look at successful transactions
        # and see if we need to send SMS
        if any(txn['success'] and txn['value'] > whale_alert_threshold for txn in transactions):
            body = f'{token} WHALE ALERT! {addr} had a transaction of more than {whale_alert_threshold} ETH.'
            message = client.messages.create(body=body, from_='+18479614395', to='+14124650602‬')
            print(body)
            

{'error': {'code': 429, 'message': 'Request limit is reached for your API key'}}
{'error': {'code': 429, 'message': 'Request limit is reached for your API key'}}
{'error': {'code': 429, 'message': 'Request limit is reached for your API key'}}
